<a href="https://colab.research.google.com/github/Faithchemurgor9/FAITH-LIMO/blob/main/End_to_end_ML_Workflow_%E2%80%94_California_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FAITH CHEMURGOR LIMO
DATA AND AI
CYBERSHUJAA
CS-DA02-25110

In [1]:
#  Imports and environment setup
import pandas as pd
import numpy as np
import pickle
import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error
import os




In [2]:
# 1) Load dataset
X, y = fetch_california_housing(return_X_y=True, as_frame=True)
print("Features:", list(X.columns))
print("Shape X:", X.shape, "y:", y.shape)

# Quick exploration (show head and summary stats)
display(X.head())
display(X.describe().T)


Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Shape X: (20640, 8) y: (20640,)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


,count,mean,std,min,25%,50%,75%,max
MedInc,20640.0,3.870671,1.899822,0.499900,2.563400,3.534800,4.743250,15.000100
HouseAge,20640.0,28.639486,12.585558,1.000000,18.000000,29.000000,37.000000,52.000000
AveRooms,20640.0,5.429000,2.474173,0.846154,4.440716,5.229129,6.052381,141.909091
AveBedrms,20640.0,1.096675,0.473911,0.333333,1.006079,1.048780,1.099526,34.066667
Population,20640.0,1425.476744,1132.462122,3.000000,787.000000,1166.000000,1725.000000,35682.000000
AveOccup,20640.0,3.070655,10.386050,0.692308,2.429741,2.818116,3.282261,1243.333333
Latitude,20640.0,35.631861,2.135952,32.540000,33.930000,34.260000,37.710000,41.950000
Longitude,20640.0,-119.569704,2.003532,-124.350000,-121.800000,-118.490000,-118.010000,-114.310000


In [3]:
# 2) Train-test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)


Train shape: (16512, 8) Test shape: (4128, 8)


In [4]:
# 3) Preprocessing pipeline - all features are numeric
numeric_features = X.columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # handle any missing values
    ('scaler', StandardScaler())                   # standardize features
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features)
])


In [5]:
# 4) Full pipeline: preprocessing + KNeighborsRegressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', KNeighborsRegressor())
])


In [6]:
# 5) Hyperparameter grid for GridSearchCV
param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1, 2]   # 1 -> Manhattan, 2 -> Euclidean
}


In [7]:
# 6) GridSearchCV with 5-fold CV, scoring R^2
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    verbose=2,
    n_jobs=-1,
    return_train_score=True
)

# 7) Fit on training data
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['MedInc',
                                                                          'HouseAge',
                                                                          'AveRooms',
                                                                          'AveBedrms',
                                                                          'Population',
                                                                          'AveOccup',
                                                                          'Latitude',
                                                                          'Longitude'])])),
                                       ('knn', KNeighborsRegressor())]),
             n_jobs=-1,
             param_grid={'knn__n_neighbors': [3, 5, 7, 9], 'knn__p': [1, 2],
                         'knn__weights': ['uniform', 'distance']},
             return_train_score=True, scoring='r2', verbose=2)

In [8]:
# 8) Results: best params, CV scores
print("Best Parameters:", grid_search.best_params_)
print("Best CV R² Score:", grid_search.best_score_)

# show a small table of the CV results
cv_results_df = pd.DataFrame(grid_search.cv_results_).sort_values(by='rank_test_score')
display(cv_results_df[['params','mean_test_score','std_test_score','mean_train_score','rank_test_score']].head(10))


Best Parameters: {'knn__n_neighbors': 9, 'knn__p': 1, 'knn__weights': 'distance'}
Best CV R² Score: 0.731266870986164


,params,mean_test_score,std_test_score,mean_train_score,rank_test_score
13,"{'knn__n_neighbors': 9, 'knn__p': 1, 'knn__wei...",0.731267,0.003097,1.000000,1
9,"{'knn__n_neighbors': 7, 'knn__p': 1, 'knn__wei...",0.729590,0.002959,1.000000,2
12,"{'knn__n_neighbors': 9, 'knn__p': 1, 'knn__wei...",0.726708,0.002832,0.782405,3
8,"{'knn__n_neighbors': 7, 'knn__p': 1, 'knn__wei...",0.725730,0.002272,0.795424,4
5,"{'knn__n_neighbors': 5, 'knn__p': 1, 'knn__wei...",0.724431,0.005392,1.000000,5
4,"{'knn__n_neighbors': 5, 'knn__p': 1, 'knn__wei...",0.721714,0.004978,0.814056,6
1,"{'knn__n_neighbors': 3, 'knn__p': 1, 'knn__wei...",0.697983,0.006403,1.000000,7
15,"{'knn__n_neighbors': 9, 'knn__p': 2, 'knn__wei...",0.697230,0.005236,1.000000,8
0,"{'knn__n_neighbors': 3, 'knn__p': 1, 'knn__wei...",0.695842,0.005695,0.849637,9
11,"{'knn__n_neighbors': 7, 'knn__p': 2, 'knn__wei...",0.695755,0.006120,1.000000,10


In [10]:
# 9) Evaluate best model on test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Test R² Score: {r2:.4f}")
print(f"Test MSE: {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")

Test R² Score: 0.7221
Test MSE: 0.3642
Test RMSE: 0.6034


In [15]:
# 10 Create a directory to save the models
SAVE_DIR = 'models'
os.makedirs(SAVE_DIR, exist_ok=True)

# 10) Save pipeline to disk (pickle & joblib examples)

pickle_path = os.path.join(SAVE_DIR, "california_knn_pipeline.pkl")
joblib_path = os.path.join(SAVE_DIR, "california_knn_pipeline.joblib")

# Save with pickle
with open(pickle_path, 'wb') as f:
    pickle.dump(best_model, f)
print("Saved pipeline with pickle ->", pickle_path)

# Save with joblib
joblib.dump(best_model, joblib_path)
print("Saved pipeline with joblib ->", joblib_path)

Saved pipeline with pickle -> models/california_knn_pipeline.pkl
Saved pipeline with joblib -> models/california_knn_pipeline.joblib


In [13]:
# 11) Example: load the pickle and test a single prediction
with open(pickle_path, 'rb') as f:
    loaded_model = pickle.load(f)

# take first 3 rows from test set as JSON-like input
sample_X = X_test.iloc[:3]
print("Sample inputs:")
display(sample_X)
print("Predictions:", loaded_model.predict(sample_X))
print("Ground truth:", y_test.iloc[:3].values)


Sample inputs:


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
20046,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01
3024,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46
15663,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44


Predictions: [0.50997325 0.93723566 4.42174044]
Ground truth: [0.477   0.458   5.00001]
